This notebook was designed to train the BoT-IoT dataset short version using Flower distributed framework

In [1]:
# we naturally first need to import torch and torchvision
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST
from typing import List, Tuple
from flwr.common import Metrics
import pandas as pd

import flwr as fl
from flwr.common import Metrics

fold1_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 1 train.csv')
fold2_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 2 train.csv')
fold3_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 3 train.csv')
fold4_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 4 train.csv')
fold5_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 5 train.csv')

fold1_test=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 1 test.csv')
fold2_test=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 2 test.csv')
fold3_test=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 3 test.csv')
fold4_test=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 4 test.csv')
fold5_test=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 5 test.csv')


C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\1855755082.py:13: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  fold1_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 1 train.csv')
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\1855755082.py:14: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  fold2_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separated classes/Folds/Fold 2 train.csv')
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\1855755082.py:15: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  fold3_train=pd.read_csv('C:/RICARDO/2023 CISTECH BACKUP/cistech/2023/Short version IoT/Only 5 folds- all classes in each node/Separate

In [2]:
DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.0.1+cu118 and Flower 1.4.0


In [3]:
final_array= ['stime','flgs_number', 'proto_number', 'sport', 'dport', 'pkts', 'bytes', 'state_number', 'ltime','seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts','sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP','TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport','AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP','N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport','Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP','category']
print(len(final_array))    

38


In [4]:
#this function transform a hexadecimal to decimal
def transform_hex_to_dec(column):
    hex_notation='x'
    dport_conversion=[]
    for x in column:
        if isinstance(x, str):   #verify if the values is string
            if hex_notation in x:   #hexadecimal values starts with 0x. it is necessary to verify if we have x in string
                dport_conversion.append(int(x, 16))
            else:                   #no x in string; thus, it is a value
                dport_conversion.append(int(x))
        else:
            dport_conversion.append(x)
    df_dport_conversion=pd.DataFrame(dport_conversion)
    return df_dport_conversion

In [5]:
from torch.utils.data import TensorDataset, DataLoader
batch_size=512

def preprocess_folds(dataset_all,batch_size):
    final_array= ['stime','flgs_number', 'proto_number', 'sport', 'dport', 'pkts', 'bytes', 'state_number', 'ltime','seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts','sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP','TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport','AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP','N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport','Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP','category']
    dataset_all=dataset_all[final_array] #filter some fields
    
    dataset_all['sport_number']=transform_hex_to_dec(dataset_all['sport'])
    dataset_all['dport_number']=transform_hex_to_dec(dataset_all['dport'])
    dataset_all['category'].replace(['DDoS','DoS','Reconnaissance','Theft','Normal'],[0,1,2,3,4], inplace=True) 
   
    final_array= ['stime','flgs_number', 'proto_number', 'sport_number', 'dport_number', 'pkts', 'bytes', 'state_number', 'ltime','seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts','sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP','TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport','AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP','N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport','Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP','category']
    dataset_all=dataset_all[final_array]
    #print(dataset_all)
    features=dataset_all.iloc[:,:-1] #filter only features since 0 and 1 contains number of row and label respectively
    print(features)
    labels=dataset_all.iloc[:,-1] #extract labels
    print(labels)

    features_np=features.to_numpy() #transform to numpy array
    features_torch=torch.from_numpy(features_np).float() #transform numpy to torch

    labels_np=labels.to_numpy() #transform array to numpy
    labels_torch=torch.tensor(labels_np)#transform numpy to tensor
    
    dataset_tensor = TensorDataset(features_torch, labels_torch) #concatenate features and labels
    #print(dataset_tensor)
    loader= DataLoader(dataset_tensor, batch_size=batch_size)  #create dataloader
    return loader

BATCH_SIZE_TRAIN=512
BATCH_SIZE_TEST=512
loader_train_fold1=preprocess_folds(fold1_train,BATCH_SIZE_TRAIN) 
loader_test_fold1=preprocess_folds(fold1_test,BATCH_SIZE_TEST)
loader_train_fold2=preprocess_folds(fold2_train,BATCH_SIZE_TRAIN) 
loader_test_fold2=preprocess_folds(fold2_test,BATCH_SIZE_TEST)
loader_train_fold3=preprocess_folds(fold3_train,BATCH_SIZE_TRAIN) 
loader_test_fold3=preprocess_folds(fold3_test,BATCH_SIZE_TEST)
loader_train_fold4=preprocess_folds(fold4_train,BATCH_SIZE_TRAIN) 
loader_test_fold4=preprocess_folds(fold4_test,BATCH_SIZE_TEST)
loader_train_fold5=preprocess_folds(fold5_train,BATCH_SIZE_TRAIN) 
loader_test_fold5=preprocess_folds(fold5_test,BATCH_SIZE_TEST)

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['sport_number']=transform_hex_to_dec(dataset_all['sport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['dport_number']=transform_hex_to_dec(dataset_all['dport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

               stime  flgs_number  proto_number  sport_number  dport_number  \
0       1.528096e+09            2             1         39807            80   
1       1.528099e+09            1             3          5777            80   
2       1.528096e+09            2             1         12999            80   
3       1.528096e+09            2             1         31449            80   
4       1.528099e+09            1             3         60131            80   
...              ...          ...           ...           ...           ...   
513657  1.529381e+09            1             1         54780          4433   
513658  1.529381e+09            1             1         54803          4433   
513659  1.529381e+09            1             1         35028            22   
513660  1.529284e+09            1             2            -1            -1   
513661  1.529381e+09            1             1         54766          4433   

        pkts  bytes  state_number         ltime    

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['sport_number']=transform_hex_to_dec(dataset_all['sport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['dport_number']=transform_hex_to_dec(dataset_all['dport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

               stime  flgs_number  proto_number  sport_number  dport_number  \
0       1.528099e+09            1             3          4440            80   
1       1.528096e+09            2             1         20574            80   
2       1.528096e+09            2             1         19800            80   
3       1.528096e+09            2             1         20489            80   
4       1.528099e+09            1             3         23080            80   
...              ...          ...           ...           ...           ...   
220137  1.529381e+09            1             1         42970          4433   
220138  1.529381e+09            1             1         35040            22   
220139  1.529381e+09            1             1         54813          4433   
220140  1.529381e+09            1             2            -1            -1   
220141  1.529381e+09            1             1         42960          4433   

        pkts  bytes  state_number         ltime    

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['sport_number']=transform_hex_to_dec(dataset_all['sport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['dport_number']=transform_hex_to_dec(dataset_all['dport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

...

               stime  flgs_number  proto_number  sport_number  dport_number  \
0       1.528099e+09            1             3         47405            80   
1       1.528099e+09            1             3         26287            80   
2       1.528096e+09            2             1         17300            80   
3       1.528096e+09            2             1         10691            80   
4       1.528096e+09            1             1         19964            80   
...              ...          ...           ...           ...           ...   
220134  1.529381e+09            1             1         54786          4433   
220135  1.529381e+09            1             1         35044            22   
220136  1.529381e+09            1             1         54799          4433   
220137  1.529381e+09            1             1         35070            22   
220138  1.529381e+09            1             1         35038            22   

        pkts  bytes  state_number         ltime    

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['sport_number']=transform_hex_to_dec(dataset_all['sport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['dport_number']=transform_hex_to_dec(dataset_all['dport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

               stime  flgs_number  proto_number  sport_number  dport_number  \
0       1.528096e+09            2             1         15835            80   
1       1.528096e+09            2             1          6624            80   
2       1.528096e+09            2             1         28573            80   
3       1.528099e+09            1             3         42330            80   
4       1.528099e+09            1             3         39382            80   
...              ...          ...           ...           ...           ...   
513653  1.529381e+09            1             1         35052            22   
513654  1.529381e+09            1             1         35048            22   
513655  1.529381e+09            1             1         54795          4433   
513656  1.529381e+09            1             1         54778          4433   
513657  1.529381e+09            1             1         35050            22   

        pkts  bytes  state_number         ltime    

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['sport_number']=transform_hex_to_dec(dataset_all['sport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_all['dport_number']=transform_hex_to_dec(dataset_all['dport'])
C:\Users\Ricardo\AppData\Local\Temp\ipykernel_50556\574589419.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

               stime  flgs_number  proto_number  sport_number  dport_number  \
0       1.528096e+09            2             1         37383            80   
1       1.528096e+09            2             1          5183            80   
2       1.528099e+09            1             3         24842            80   
3       1.528099e+09            1             3          7499            80   
4       1.528099e+09            1             3         64869            80   
...              ...          ...           ...           ...           ...   
220133  1.526350e+09            6             3         44417         64513   
220134  1.529284e+09            1             1         36682            22   
220135  1.529381e+09            1             1         54770          4433   
220136  1.529381e+09            1             1         35036            22   
220137  1.529381e+09            1             1         42989          4433   

        pkts  bytes  state_number         ltime    

In [6]:
trainloaders= [loader_train_fold1,loader_train_fold2,loader_train_fold3,loader_train_fold4,loader_train_fold5]
valloaders = [loader_test_fold1,loader_test_fold2,loader_test_fold3,loader_test_fold4,loader_test_fold5]

# A CNN architecture

Thist tutorial is not so much about novel architectural designs so we keep things simple and make use of a typical CNN that is adequate for the MNIST image classification taks.



In [7]:
import torch.nn as nn
import torch.nn.functional as F
DIM_IN = 37

DIM_OUT = 5
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = torch.nn.Linear(DIM_IN, 100)
        self.relu = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(100, 100 )
        self.relu2 = torch.nn.ReLU()
        self.layer3 = torch.nn.Linear(100, 100 )
        self.relu3 = torch.nn.ReLU()
        self.layer4 = torch.nn.Linear(100, DIM_OUT )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.layer3(x)
        x = self.relu3(x)
        x = self.layer4(x)
        
        return x

Similarly to what we did with the dataset you could inspect the model in various ways. We can, for instance, count the number of model parameters.

In [8]:
model = Net()
num_parameters = sum(value.numel() for value in model.state_dict().values())
print(f"{num_parameters = }")

num_parameters = 24505


## The Training Loop

A minimal training loop in PyTorch can be constructed with three functions:
*  `train()` that will train the model given a dataloader.
* `test()` that will be used to evaluate the performance of the model on held-out data, e.g., a training set.
* `run_centralised()` which will define additional elements (e.g. the optimiser) and run the training loop over N epochs.

Let's construct these functions!


In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")
   
def test(net, testloader):
    """Validate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, loss = 0, 0.0
    #Ricardo Manzano. Evaluate f1-score for each class 
    true_values=[]
    predicted_values=[]
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            #Ricardo Manzano
            pred = outputs.max(1).indices      # find the index of the max value
            pred_numpy=pred.data.cpu().numpy()    #transform prediction to numpy from gpu
            labels_numpy=labels.data.cpu().numpy()  #transform labels to numpy from gpu
            true_values.extend(labels_numpy)
            predicted_values.extend(pred_numpy)
            
            #----------------
            
            correct += (predicted == labels).sum().item()
    #Ricardo Manzano. Confusion Matrix
  
    target_names=['0','1','2','3','4']  
    # cf_matrix = confusion_matrix(true_values, predicted_values)  #find confusion matrix
    # print(cf_matrix)
    report=classification_report( true_values, predicted_values,target_names=target_names,zero_division=0,output_dict=True)
    #print(report)
    macro_f1 = report['macro avg']['f1-score']
    f1_class_0 = report['0']['f1-score']
    f1_class_1 = report['1']['f1-score']
    f1_class_2 = report['2']['f1-score']
    f1_class_3 = report['3']['f1-score']
    f1_class_4 = report['4']['f1-score']
  
 
    accuracy = correct / len(testloader.dataset)
    return loss, accuracy,macro_f1,f1_class_0, f1_class_1, f1_class_2, f1_class_3, f1_class_4


    

Let's run this for 5 epochs (you'll see it reaching close to 99% accuracy -- as expected from a centralised setup with the MNIST dataset)

The above centralised formulation has worked just fine for some applications and to showcase the potential of AI in a variety of scenarios. However, as was discussed earlier in the session, centralised training is unsuitable for a larger range of settings were information cannot be collected in order to build a unified (centralised) dataset.

# The Future of AI is Federated

What are the key differences between Federated Learning and Centralised Training? I you could only pick you, probably you'd say:
* Federated Learning is distributed -- the model is trained on-device by the participating clients.
* Data remains private and is owned by a specific _client_ -- the data is never sent to the central server.

The are several more differences. But the above two are the main ones to always consider and that are common to all flavours of Federated Learning (e.g. _cross-device_ or _cross-silo_). The remaining of this tutorial is going to focus in transforming the code we have written so far for the centralised setting and construct a Federated Learning pipeline using Flower and PyTorch.

Let's begin! 🚀

## One Client, One Data Partition

To start designing a Federated Learning pipeline we need to meet one of the key properties in FL: each client has its own data partition. To acomplish this with the MNIST dataset, we are going to generate N random paritions, where N is the total number of clients in our FL system.

Now let's defice our Flower Client class:

In [10]:
import numpy as np
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [11]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        

        loss, accuracy,macro_f1,f1_class_0, f1_class_1, f1_class_2, f1_class_3, f1_class_4 = test(self.net, self.valloader) # <-------------------------- calls the `test` function, just what we did in the centralised setting (but this time using the client's local validation set)
        
        # send statistics back to the server
        return float(loss), len(self.valloader), {'accuracy': accuracy,'macro_f1':macro_f1,'f1_class_0':f1_class_0,'f1_class_1': f1_class_1,'f1_class_2': f1_class_2,'f1_class_3': f1_class_3,'f1_class_4': f1_class_4}

In [12]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)

In [13]:
#This function is used to obtain the values of accuracy when we evaluate the global model in validation local dataset
#We need to define how aggregate these accuracies

#the evaluate_function return the following   
#return float(loss), len(self.valloader), {'accuracy': accuracy}
# As we can see, it returns the len of the validation dataloader and the accuracy 
# we need to average the accuracy of the validation dataset of each local 

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]  #append the accuracy value for each local validation
    macro_f1 = [num_examples * m["macro_f1"] for num_examples, m in metrics]  #append the accuracy value for each local validation
    f1_class_0 = [num_examples * m["f1_class_0"] for num_examples, m in metrics]  #append the accuracy value for each local validation
    f1_class_1 = [num_examples * m["f1_class_1"] for num_examples, m in metrics]  #append the accuracy value for each local validation
    f1_class_2 = [num_examples * m["f1_class_2"] for num_examples, m in metrics]  #append the accuracy value for each local validation
    f1_class_3 = [num_examples * m["f1_class_3"] for num_examples, m in metrics]  #append the accuracy value for each local validation
    f1_class_4 = [num_examples * m["f1_class_4"] for num_examples, m in metrics]  #append the accuracy value for each local validation
   
    examples = [num_examples for num_examples, _ in metrics]  #append the length of each local validation

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples),"macro_f1": sum(macro_f1 ) / sum(examples),"f1_class_0": sum(f1_class_0) / sum(examples),"f1_class_1": sum(f1_class_1) / sum(examples),"f1_class_2": sum(f1_class_2) / sum(examples),"f1_class_3": sum(f1_class_3) / sum(examples),"f1_class_4": sum(f1_class_4) / sum(examples)}  # sum up all the accuracies in each node and divided for all number of datasamples sum

In [14]:
# Create FedAvg strategy
NUM_CLIENTS = 5
from collections import OrderedDict
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1,  # Sample 50% of available clients for evaluation
    min_fit_clients=3,  # Never sample less than 10 clients for training
    min_evaluate_clients=3,  # Never sample less than 5 clients for evaluation
    min_available_clients=3,  # Wait until all 10 clients are available
    evaluate_metrics_aggregation_fn=weighted_average
    
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
history=fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=100),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-07-21 13:36:54,836 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
2023-07-21 13:37:00,160	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-21 13:37:02,579 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2099860684.0, 'node:127.0.0.1': 1.0, 'memory': 4199721371.0, 'CPU': 20.0, 'GPU': 1.0}
INFO flwr 2023-07-21 13:37:02,581 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-21 13:37:02,582 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-21 13:37:09,809 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-21 13:37:09,809 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-07-21 13:37:09,809 | server.py:101 | FL starting
DEBUG flwr 2023-07-21 13:37:09,809 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)
DEBUG flwr 2023-07-21 13:38:03,274 | server.py:

Spend a few minutes to inspect the `FlowerClient` class above. Please ask questions if there is something unclear !

Then keeneyed among you might have realised that if we were to fuse the client's `fit()` and `evaluate()` methods, we'll end up with essentially the same as in the `run_centralised()` function we used in the Centralised Training part of this tutorial. And it is true!! In Federated Learning, the way clients perform local training makes use of the same principles as more traditional centralised setup. The key difference is that the dataset now is much smaller and it's never _"seen"_ by the entity running the FL workload (i.e. the central server).


Talking baout the central server... we should define what strategy we want to make use of so the updated models sent from the clients back to the server at the end of the `fit()` method are aggregate.


## Chosing a Flower Strategy


A strategy sits at the core of the Federated Learining experiment. It is involved in all stages of a FL pipeline: sampling clients; sending the _global model_ to the clients so they can do `fit()`; receive the updated models from the clients and **aggregate** these to construct a new _global model_; define and exectue global or federated evaluation; and more.

Flower comes with [many strategies built-in](https://github.com/adap/flower/tree/main/src/py/flwr/server/strategy) and more to be avaialable in the next release (`1.5` already!). For this tutorial, let's use what is arguable the most popular strategy out there: `FedAvg`.

The way `FedAvg` works is simple but performs surprisingly well in practice. It is therefore one good strategy to start your experimentation. `FedAvg`, as its name implies, derieves a new version of the _global model_ by taking the average of all the models sent by clients participating in the round. You can read all the details [in the paper](https://arxiv.org/abs/1602.05629).

Let's see how we can define `FedAvg` using Flower. We use one of the callbacks called `evaluate_fn` so we can easily evaluate the state of the global model using a small centralised testset. Note this functionality is user-defined since it requires a choice in terms of ML-framework. (if you recall, Flower is framework agnostic).

> This being said, centralised evaluation of the global model is only possible if there exists a centralised dataset that somewhat follows a similar distribution as the data that's spread across clients. In some cases having such centralised dataset for validation is not possible, so the only solution is to federate the evaluation of the _global model_. This is the default behaviour in Flower. If you don't specify teh `evaluate_fn` arguemtn in your strategy, then, centralised global evaluation won't be perfromed.

In [ ]:
print(f"{history.metrics_distributed = }")

Doing 10 rounds should take less than 2 minutes on a CPU-only Colab instance <-- Flower Simulation is fast! 🚀

You can then use the resturned `History` object to either save the results to disk or do some visualisation (or both of course, or neither if you like chaos). Below you can see how you can plot the centralised accuracy obtainined at the end of each round (including at the very beginning of the experiment) for the _global model_. This is want the function `evaluate_fn()` that we passed to the strategy reports.

In [ ]:
#the output history can have 4 components
#loss centralized: no need to aggregate, just a single value
#metrics centralized: no need to aggregate, just a single value for each key  # code: history.metrics_centralized
#loss distributed: can be automatically aggregated, strategy knows how
#metrics distributed: must be aggregated, but can not be done automatica   # code: history.metrics_distributed
# We are evaluating 
#print(f"{history.metrics_distributed = }")  

def plot_results(metric,label1):
    round = [data[0] for data in metric]
    acc = [100.0*data[1] for data in metric]
    plt.plot(round, acc, label=label1)
    plt.grid()
    plt.ylabel('Accuracy (%)')
    plt.xlabel('Round')
    plt.title('MNIST - IID - 5 clients')
    print(label1)
    plt.legend()

results_array=['macro_f1','f1_class_0','f1_class_1','f1_class_2','f1_class_3','f1_class_4','f1_class_5','f1_class_6','f1_class_7','f1_class_8','f1_class_9']
for i in results_array:
    global_accuracy_centralised = history.metrics_distributed[i]
    plot_results(global_accuracy_centralised,i)
